In [8]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json

In [9]:
user_agent = {'User-agent': 'Mozilla/5.0'}

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [10]:
def get_film_data(film_id):
    
    delimiter = '+_+'
    
    film_url = "http://www.imdb.com/title/" + film_id
    driver.get(film_url)
    
    response = requests.get(film_url, headers = user_agent)
    html = response.text
    soup = bs(html, 'lxml')
    
    film_dict = {}
    
    try:
        genre_selector = "//*[@id='titleStoryLine']//a[contains(@href, '/genre/')]"
        film_dict['genres'] = ''
        for genre in driver.find_elements_by_xpath(genre_selector):
            film_dict['genres'] = film_dict['genres'] + delimiter + genre.text
    except NoSuchElementException:
        film_dict['genres'] = None
        
    try:
        title_selector = "//*[@id='title-overview-widget']//*[@itemprop='name']"
        film_dict['title'] = driver.find_element_by_xpath(title_selector).text
    except NoSuchElementException:
        film_dict['title'] = None
    
    try:
        year_selector = '//*[@id="titleYear"]'
        film_dict['year'] = driver.find_element_by_xpath(year_selector).text.split('(')[1].split(')')[0]
    except NoSuchElementException:
        film_dict['year'] = None  
    
    try:
        rating_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingValue']"
        film_dict['rating'] = driver.find_element_by_xpath(rating_selector).text
    except NoSuchElementException:
        film_dict['rating'] = None  
   
    try:
        votes_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingCount']"
        film_dict['votes'] = driver.find_element_by_xpath(votes_selector).text
    except NoSuchElementException:
        film_dict['votes'] = None     
    
    try:
        directors_selector = "//*[@id='title-overview-widget']//*[@itemprop='director']"
        film_dict['directors'] = ''
        for director in driver.find_elements_by_xpath(directors_selector):
            film_dict['directors'] = film_dict['directors'] + delimiter + director.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['directors'] = None    
  
    try:
        writers_selector = "//*[@id='title-overview-widget']//*[@itemprop='creator']"
        film_dict['writers'] = ''
        for writer in driver.find_elements_by_xpath(writers_selector):
            film_dict['writers'] = film_dict['writers'] + delimiter + writer.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['writers'] = None     

    try:
        actors_selector = "//*[@id='titleCast']//*[@itemprop='name']"
        film_dict['actors'] = ''
        for actor in driver.find_elements_by_xpath(actors_selector):
            film_dict['actors'] = film_dict['actors'] + delimiter + actor.text
    except NoSuchElementException:
        film_dict['actors'] = None          

    try:
        keywords_selector = "//*[@class='see-more inline canwrap']//*[@itemprop='keywords']"
        film_dict['keywords'] = ''
        for keyword in driver.find_elements_by_xpath(keywords_selector):
            film_dict['keywords'] = film_dict['keywords'] + delimiter + keyword.text
    except NoSuchElementException:
        film_dict['keywords'] = None       

    try:
        mpaa_selector = "//meta[@itemprop='contentRating']"
        film_dict['mpaa'] = driver.find_element_by_xpath(mpaa_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['mpaa'] = None      

    try:
        release_date_selector = "//meta[@itemprop='datePublished']"
        film_dict['release_date'] = driver.find_element_by_xpath(release_date_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['release_date'] = None     

    try:
        languages_selector = "//*[@id='titleDetails']//a[contains(@href, 'language')]"
        film_dict['languages'] = ''
        for language in driver.find_elements_by_xpath(languages_selector):
            film_dict['languages'] = film_dict['languages'] + delimiter + language.text
    except NoSuchElementException:
        film_dict['languages'] = None   
    
    try:
        countries_selector = "//*[@id='titleDetails']//a[contains(@href, 'country_of_origin')]"
        film_dict['countries'] = ''
        for country in driver.find_elements_by_xpath(countries_selector):
            film_dict['countries'] = film_dict['countries'] + delimiter + country.text
    except NoSuchElementException:
        film_dict['countries'] = None 
    
    try:
        runtime_selector = "//*[@id='titleDetails']//*[@itemprop='duration']"
        film_dict['runtime'] = int(driver.find_element_by_xpath(runtime_selector).text.split(' ')[0])
    except NoSuchElementException:
        film_dict['runtime'] = None 
     
    film_dict['budget'] = None
    film_dict['gross_usa'] = None
    for h4 in soup.find_all('h4'):
        if "Budget:" in h4:
            film_dict['budget'] = h4.next_sibling        
        if "Gross USA:" in h4:
            film_dict['gross_usa'] = h4.next_sibling
            
    return film_dict

In [11]:
with open("film_ids.json","r") as f:
      data = f.read()
        
film_ids = json.loads(data)

In [12]:
dict_list = []
dummy = 0

for film_id in film_ids[1000:2000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
import pandas as pd

data = json.dumps(dict_list)

with open("film_data_2.json","w") as f:
      f.write(data)

with open("film_data_2.json","r") as f:
      data = f.read()
        
d = json.loads(data)

df = pd.DataFrame(d)

df

,actors,budget,countries,directors,genres,gross_usa,keywords,languages,mpaa,rating,release_date,runtime,title,votes,writers,year
0,+_+Dennis Farina+_+Michael Worth+_+Patricia Ze...,None,+_+USA,+_+Stephen Smoke,+_+Action+_+Drama,None,+_+drugs+_+neighborhood+_+failed kidnapping+_+...,+_+English,R,5.4,1992-09-02,NaN,Street Crimes (1992),121,+_+Stephen Smoke,1992
1,+_+Clint Eastwood+_+Gene Hackman+_+Morgan Free...,"$14,400,000\n",+_+USA,+_+Clint Eastwood,+_+Drama+_+Western,"$101,157,447\n",+_+one last job+_+leaving flowers on a grave+_...,+_+English,R,8.2,1992-08-07,130.0,Unforgiven (1992),"310,855",+_+David Webb Peoples,1992
2,+_+Gregory Harrison+_+Lisa Pescia+_+Morton Dow...,None,+_+USA,+_+Adam Simon,+_+Drama+_+Thriller,None,+_+erotic thriller+_+blindfold+_+bare chested ...,+_+English,R,3.8,1992-08-05,84.0,Body Chemistry II: The Voice of a Stranger (1992),194,"+_+Jackson Barr,+_+Christopher Wooden",1992
3,+_+Elliott Gould+_+Emilia Crow+_+Karen Black+_...,None,+_+USA,+_+William Sachs,+_+Drama+_+Music+_+Thriller,None,+_+judge+_+juvenile court+_+female nudity+_+de...,+_+English,R,4.6,1992-08-05,85.0,Judgement (1992),83,+_+William Sachs,1992
4,+_+Victor Wong+_+Michael Treanor+_+Max Elliott...,"$6,500,000\n",+_+USA,+_+Jon Turteltaub,+_+Action+_+Comedy+_+Family+_+Sport,"$29,000,301\n",+_+grandfather+_+brother+_+ninja+_+fbi+_+marti...,+_+English,PG,5.2,1992-08-07,84.0,3 Ninjas (1992),"17,691",+_+Kenny Kim+_+Edward Emanuel,1992
5,+_+John Lithgow+_+Lolita Davidovich+_+Steven B...,"$12,000,000\n",+_+USA,+_+Brian De Palma,+_+Crime+_+Drama+_+Horror+_+Thriller,"$21,370,057\n",+_+psychologist+_+wig+_+long tracking shot+_+p...,+_+English+_+Spanish,R,6.0,1992-08-07,91.0,Raising Cain (1992),"11,201",+_+Brian De Palma,1992
6,+_+Annabella Sciorra+_+Jamey Sheridan+_+Anthon...,"$18,000,000\n",+_+USA,+_+Christopher Crowe,+_+Drama+_+Mystery+_+Thriller,"$11,124,511\n",+_+psychiatrist+_+new york city+_+erotic thril...,+_+English,R,5.6,1992-08-07,103.0,Whispers in the Dark (1992),"1,699",+_+Christopher Crowe,1992
7,+_+Cynthia Basinet+_+Elaine Hendrix+_+Kurt T. ...,None,+_+USA,+_+Tony Markes,+_+Thriller,None,,+_+English,None,5.8,1992-08-16,83.0,Last Dance (1992),195,+_+Emerson Bixby,1992
8,+_+Sheila Armstrong+_+Jake Azer+_+Mielle Conkl...,$100\n,+_+USA,+_+Jonathan Morrill,+_+Comedy,None,,+_+English,None,None,1992-08-08,50.0,Jailbait Summer (1992),None,"+_+Frank Crivelli,+_+Jonathan Morrill",1992
9,+_+Bridget Fonda+_+Jennifer Jason Leigh+_+Stev...,"$16,000,000\n",+_+USA,+_+Barbet Schroeder,+_+Drama+_+Thriller,"$47,922,919\n",+_+female frontal nudity+_+male nudity+_+twin+...,+_+English,R,6.3,1992-08-14,107.0,Single White Female (1992),"26,334",+_+John Lutz+_+Don Roos,1992


In [18]:
df.loc[512]

actors          +_+Daphne Zuniga+_+Tom Verica+_+Barry Bostwick...
budget                                                       None
countries                                           +_+Peru+_+USA
directors                                           +_+Luis Llosa
genres                                      +_+Action+_+Adventure
gross_usa                                                    None
keywords                     +_+based on novel+_+independent film
languages                                              +_+English
mpaa                                                        PG-13
rating                                                        4.7
release_date                                           1993-07-21
runtime                                                       NaN
title                Eight Hundred Leagues Down the Amazon (1993)
votes                                                         158
writers                           +_+Jackson Barr,+_+Laura Schiff
year      